In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install the pyspark
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 199 kB 61.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c6ef6011e8fc907a2be2c8d1824a87a0546fbfdc8821760152423a016e9071ac
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql import functions as f, udf
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import col,isnan, when, count
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [ ]:
# create the session
conf = pyspark.SparkConf()
# create the context
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sqlcontext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


## Linear Regression

In [ ]:
# read the data and display
data = spark.read.csv('/content/drive/MyDrive/13-07-2022/Ecommerce Customers.csv',header='true')
data.show(5)

+--------------------+--------------------+---------+-------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg. Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+-------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|    835 Frank Tunnel|     null|               null|              null|              null|                null|               null|
|         Wrightmouth|      MI 82180-9605"|   Violet|  34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|
|   hduke@hotmail.com|  4547 Archer Common|     null|               null|              null|              null|                null|               null|
|         Diazchester|      CA 06566-8576"|DarkGreen| 31.926272026360156|11.109460

In [ ]:
data = data.withColumnRenamed("Avg. Session Length","Avg Session Length")

In [ ]:
data.show(5)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|    835 Frank Tunnel|     null|              null|              null|              null|                null|               null|
|         Wrightmouth|      MI 82180-9605"|   Violet| 34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|
|   hduke@hotmail.com|  4547 Archer Common|     null|              null|              null|              null|                null|               null|
|         Diazchester|      CA 06566-8576"|DarkGreen|31.926272026360156|11.1094607286825

In [ ]:
# find the null values in the data
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|Email|Address|Avatar|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|    0|      0|   500|               500|        500|            500|                 500|                546|
+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+



In [ ]:
# drop the null values
data=data.na.drop()

In [ ]:
# find the null values in the data
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|Email|Address|Avatar|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|    0|      0|     0|                 0|          0|              0|                   0|                  0|
+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+



In [ ]:
data.show(5)

+---------------+---------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|          Email|        Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+---------------+---------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|    Wrightmouth| MI 82180-9605"|          Violet| 34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|  587.9510539684005|
|    Diazchester| CA 06566-8576"|       DarkGreen|31.926272026360156|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    Cobbborough| DC 99414-7564"|          Bisque|33.000914755642675|11.330278057777512| 37.11059744212085|   4.104543202376424| 487.54750486747207|
|     Port Jason| OH 22070-1220"|     SaddleBrown| 34.30555662975554|13.717513665142508| 36.72128267790313

In [ ]:
# display the schema and the number of rows
data.printSchema()
print("Rows: %s" % data.count())

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: string (nullable = true)
 |-- Time on App: string (nullable = true)
 |-- Time on Website: string (nullable = true)
 |-- Length of Membership: string (nullable = true)
 |-- Yearly Amount Spent: string (nullable = true)

Rows: 454


In [ ]:
# final_data=data.select(,"Yearly Amount Spent")
final_data = data.select(col('Yearly Amount Spent').alias('label'),"Avg Session Length","Time on App","Time on Website","Length of Membership")
# final_data = final_data.withColumnRenamed("Yearly Amount Spent","label")

In [ ]:
final_data.show(5)

+------------------+------------------+------------------+------------------+--------------------+
|             label|Avg Session Length|       Time on App|   Time on Website|Length of Membership|
+------------------+------------------+------------------+------------------+--------------------+
| 587.9510539684005| 34.49726772511229|12.655651149166752| 39.57766801952616|   4.082620632952961|
| 392.2049334443264|31.926272026360156|11.109460728682564|37.268958868297744|    2.66403418213262|
|487.54750486747207|33.000914755642675|11.330278057777512| 37.11059744212085|   4.104543202376424|
| 581.8523440352178| 34.30555662975554|13.717513665142508| 36.72128267790313|  3.1201787827480914|
| 599.4060920457634| 33.33067252364639|12.795188551078114| 37.53665330059473|   4.446308318351435|
+------------------+------------------+------------------+------------------+--------------------+
only showing top 5 rows



In [ ]:
# type casting the data
final_data=final_data.select([col(c).cast("float") for c in final_data.columns])
final_data.show(5)

+---------+------------------+-----------+---------------+--------------------+
|    label|Avg Session Length|Time on App|Time on Website|Length of Membership|
+---------+------------------+-----------+---------------+--------------------+
|587.95105|          34.49727|  12.655651|      39.577667|           4.0826206|
|392.20493|         31.926271|  11.109461|       37.26896|           2.6640341|
|487.54752|         33.000916|  11.330278|      37.110596|            4.104543|
|581.85236|         34.305557|  13.717514|      36.721283|           3.1201787|
|599.40607|         33.330673|  12.795189|       37.53665|            4.446308|
+---------+------------------+-----------+---------------+--------------------+
only showing top 5 rows



In [ ]:
# select the columns to be used as the features (all except 'Yearly Amount Spent')
featureColumns = [c for c in final_data.columns if c != 'label']

# create and configure the assembler
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")

# transform the original data
dataDF = assembler.transform(final_data)

In [ ]:
dataDF.show(5)

+---------+------------------+-----------+---------------+--------------------+--------------------+
|    label|Avg Session Length|Time on App|Time on Website|Length of Membership|            features|
+---------+------------------+-----------+---------------+--------------------+--------------------+
|587.95105|          34.49727|  12.655651|      39.577667|           4.0826206|[34.4972686767578...|
|392.20493|         31.926271|  11.109461|       37.26896|           2.6640341|[31.9262714385986...|
|487.54752|         33.000916|  11.330278|      37.110596|            4.104543|[33.0009155273437...|
|581.85236|         34.305557|  13.717514|      36.721283|           3.1201787|[34.3055572509765...|
|599.40607|         33.330673|  12.795189|       37.53665|            4.446308|[33.3306732177734...|
+---------+------------------+-----------+---------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
final=dataDF.select("label","features")
final.show(5)

+---------+--------------------+
|    label|            features|
+---------+--------------------+
|587.95105|[34.4972686767578...|
|392.20493|[31.9262714385986...|
|487.54752|[33.0009155273437...|
|581.85236|[34.3055572509765...|
|599.40607|[33.3306732177734...|
+---------+--------------------+
only showing top 5 rows



In [ ]:
# splitting the dataset
train, test = final.randomSplit([0.7, 0.3])

In [ ]:
# feeding th data to the model
lr = LinearRegression(labelCol ='label', featuresCol ='features')

In [ ]:
# training the model
lr_model=lr.fit(train)
# predict on test data
predict_test  = lr_model.transform(test)

In [ ]:
# actual and predicted price of fare
predict_test.select("label", "prediction").show(10)

+---------+------------------+
|    label|        prediction|
+---------+------------------+
|282.47125| 285.3814482450148|
| 350.0582| 343.6190508053942|
|357.85797| 346.6243586238213|
| 357.8637|351.37317467861044|
| 368.6548|377.70015637334404|
|374.26968|369.37608010296526|
|375.39847| 375.3393046090978|
|376.49683| 391.4569629085199|
|378.47357|  372.394082127983|
|  385.095| 385.7628861496771|
+---------+------------------+
only showing top 10 rows



In [ ]:
# model summary
training_summary=lr_model.summary
print('No. of Iterations',training_summary.totalIterations)

No. of Iterations 0


In [ ]:
print('Objective History',training_summary.objectiveHistory)

Objective History [0.0]


In [ ]:
training_summary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-15.587495415130206|
| -5.221742263440774|
| -6.402189278401693|
|  18.40341150496397|
|   -9.0860256546855|
|0.02901652640412067|
| -9.753599483513426|
| -9.556044909877983|
|  5.899363630348489|
|-0.8823671822960932|
| -4.301951526254243|
| -6.068953691879187|
| 16.566074572593834|
| -6.763151799483012|
| -3.564810927678991|
| -12.15934645962534|
|-15.000046156686949|
|  7.032866432692572|
| 10.350925831443874|
| 1.3932981646339613|
+-------------------+
only showing top 20 rows



## Logistic Regression

In [ ]:
# Transform each qualititative data in the dataset into a double numeric value
def getDoubleValue(input):
    result = 0.0
    if (input == 'P'):
        result = 3.0
    if (input == 'A'):
        result = 2.0
    if (input == 'N'):
        result = 1.0
    if (input == 'NB'):
        result = 1.0
    if (input == 'B'):
        result = 0.0
    return result

In [ ]:
# read the file
data = sc.textFile("/content/drive/MyDrive/fiverr/13-07-2022/Qualitative_Bankruptcy.txt")
data.count()

250

In [ ]:
# Prepare data for the logistic regression algorithm
dataTuple=data.map(lambda o: (getDoubleValue(o.split(",")[0]),getDoubleValue(o.split(",")[1]),getDoubleValue(o.split(",")[2]),getDoubleValue(o.split(",")[3]),getDoubleValue(o.split(",")[4]),getDoubleValue(o.split(",")[5]),getDoubleValue(o.split(",")[6])))
df=dataTuple.toDF(['Industrial Risk','Management Risk','Financial Flexibility','Credibility','Competitiveness','Operating Risk','Class'])
df.show(5)

+---------------+---------------+---------------------+-----------+---------------+--------------+-----+
|Industrial Risk|Management Risk|Financial Flexibility|Credibility|Competitiveness|Operating Risk|Class|
+---------------+---------------+---------------------+-----------+---------------+--------------+-----+
|            3.0|            3.0|                  2.0|        2.0|            2.0|           3.0|  1.0|
|            1.0|            1.0|                  2.0|        2.0|            2.0|           1.0|  1.0|
|            2.0|            2.0|                  2.0|        2.0|            2.0|           2.0|  1.0|
|            3.0|            3.0|                  3.0|        3.0|            3.0|           3.0|  1.0|
|            1.0|            1.0|                  3.0|        3.0|            3.0|           1.0|  1.0|
+---------------+---------------+---------------------+-----------+---------------+--------------+-----+
only showing top 5 rows



In [ ]:
# create and configure the assembler
ignore = ['Class']
assembler = VectorAssembler(
    inputCols=[x for x in df.columns if x not in ignore],
    outputCol='features')

assembler_df = assembler.transform(df)
assembler_df.show(5)

+---------------+---------------+---------------------+-----------+---------------+--------------+-----+--------------------+
|Industrial Risk|Management Risk|Financial Flexibility|Credibility|Competitiveness|Operating Risk|Class|            features|
+---------------+---------------+---------------------+-----------+---------------+--------------+-----+--------------------+
|            3.0|            3.0|                  2.0|        2.0|            2.0|           3.0|  1.0|[3.0,3.0,2.0,2.0,...|
|            1.0|            1.0|                  2.0|        2.0|            2.0|           1.0|  1.0|[1.0,1.0,2.0,2.0,...|
|            2.0|            2.0|                  2.0|        2.0|            2.0|           2.0|  1.0|[2.0,2.0,2.0,2.0,...|
|            3.0|            3.0|                  3.0|        3.0|            3.0|           3.0|  1.0|[3.0,3.0,3.0,3.0,...|
|            1.0|            1.0|                  3.0|        3.0|            3.0|           1.0|  1.0|[1.0,1.0,3.0,3

In [ ]:
# create label column
label_df = assembler_df.withColumn("label", assembler_df["Class"])
label_df.show(5)

+---------------+---------------+---------------------+-----------+---------------+--------------+-----+--------------------+-----+
|Industrial Risk|Management Risk|Financial Flexibility|Credibility|Competitiveness|Operating Risk|Class|            features|label|
+---------------+---------------+---------------------+-----------+---------------+--------------+-----+--------------------+-----+
|            3.0|            3.0|                  2.0|        2.0|            2.0|           3.0|  1.0|[3.0,3.0,2.0,2.0,...|  1.0|
|            1.0|            1.0|                  2.0|        2.0|            2.0|           1.0|  1.0|[1.0,1.0,2.0,2.0,...|  1.0|
|            2.0|            2.0|                  2.0|        2.0|            2.0|           2.0|  1.0|[2.0,2.0,2.0,2.0,...|  1.0|
|            3.0|            3.0|                  3.0|        3.0|            3.0|           3.0|  1.0|[3.0,3.0,3.0,3.0,...|  1.0|
|            1.0|            1.0|                  3.0|        3.0|         

In [ ]:
# parse the data
parsed_data = assembler_df.rdd.map(lambda x: LabeledPoint(x[6], x[:6]))
parsed_data.take(5)

[LabeledPoint(1.0, [3.0,3.0,2.0,2.0,2.0,3.0]),
 LabeledPoint(1.0, [1.0,1.0,2.0,2.0,2.0,1.0]),
 LabeledPoint(1.0, [2.0,2.0,2.0,2.0,2.0,2.0]),
 LabeledPoint(1.0, [3.0,3.0,3.0,3.0,3.0,3.0]),
 LabeledPoint(1.0, [1.0,1.0,3.0,3.0,3.0,1.0])]

In [ ]:
# Split train & test
train_data, test_data = parsed_data.randomSplit([0.6, 0.4], seed=11)
print("Training Dataset Count : " + str(train_data.count()))
print("Test Dataset Count : " + str(test_data.count()))

Training Dataset Count : 162
Test Dataset Count : 88


In [ ]:
# fit the model
model = LogisticRegressionWithLBFGS.train(train_data, numClasses=2)

In [ ]:
# get results on test data
labels_and_preds = test_data.map(lambda p: (p.label, model.predict(p.features)))

In [ ]:
# get error
train_err = labels_and_preds.filter(lambda lp: lp[0] != lp[1]).count() / float(test_data.count())

In [ ]:
print("Training Error = " + str(train_err))

Training Error = 0.17045454545454544
